In [1]:
from pdf2image import convert_from_path, convert_from_bytes
from matplotlib import pyplot as plt
from matplotlib import image as img
from PIL import Image
import scipy.fftpack
import cv2
import numpy as np

In [2]:
def phash(image, hash_size=32, highfreq_factor=4):
    pixels = np.asarray(image)
    dct = scipy.fftpack.dct(scipy.fftpack.dct(pixels, axis=0), axis=1)
    dctlowfreq = dct[:hash_size, :hash_size]
    med = np.median(dctlowfreq)
    diff = dctlowfreq > med
    return diff.astype(np.uint16)

def drop_duplicates(images):
    SIZE = 32
    REWIND = 5
    total_len = len(images)
    hash_list = []
    for i in range(total_len):
        image = np.asarray(images[i].convert("L")).astype(np.float64)
        print("Page",str(i+1)+"/"+str(total_len), "hash obtained.")
        image = cv2.resize(image,(SIZE, SIZE))
        hash_list.append(phash(image).reshape(-1))
    return hash_list

In [3]:
fpath = 've216,chap1,teach.pdf'
images = convert_from_path(fpath)

In [4]:
hash_list = drop_duplicates(images)

Page 1/403 hash obtained.
Page 2/403 hash obtained.
Page 3/403 hash obtained.
Page 4/403 hash obtained.
Page 5/403 hash obtained.
Page 6/403 hash obtained.
Page 7/403 hash obtained.
Page 8/403 hash obtained.
Page 9/403 hash obtained.
Page 10/403 hash obtained.
Page 11/403 hash obtained.
Page 12/403 hash obtained.
Page 13/403 hash obtained.
Page 14/403 hash obtained.
Page 15/403 hash obtained.
Page 16/403 hash obtained.
Page 17/403 hash obtained.
Page 18/403 hash obtained.
Page 19/403 hash obtained.
Page 20/403 hash obtained.
Page 21/403 hash obtained.
Page 22/403 hash obtained.
Page 23/403 hash obtained.
Page 24/403 hash obtained.
Page 25/403 hash obtained.
Page 26/403 hash obtained.
Page 27/403 hash obtained.
Page 28/403 hash obtained.
Page 29/403 hash obtained.
Page 30/403 hash obtained.
Page 31/403 hash obtained.
Page 32/403 hash obtained.
Page 33/403 hash obtained.
Page 34/403 hash obtained.
Page 35/403 hash obtained.
Page 36/403 hash obtained.
Page 37/403 hash obtained.
Page 38/40

In [13]:
hash_altr = hash_list[1::]

In [25]:
diff = [15000000]
for i in range(len(hash_altr)):
    hash_diff = abs(hash_altr[i]-hash_list[i])
    hamming_dist = hash_diff.sum()
    diff.append(hamming_dist)

f = lambda i : (i>=10000000)
pages_to_keep = list(map(f,diff))
diff.append(15000000)

In [26]:
import PyPDF2

pdfReader = PyPDF2.PdfFileReader(fpath)
pdfnums = pdfReader.numPages
pdfWriter = PyPDF2.PdfFileWriter()
for num in range(pdfnums):
    if diff[num+1]>=10000000:
        pageObj = pdfReader.getPage(num)
        pdfWriter.addPage(pageObj)
with open(fpath.replace('.pdf','_modified.pdf'), 'wb') as pdfOutputFile:
    pdfWriter.write(pdfOutputFile)